In [13]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
import glob
import pandas as pd
import os
import seaborn as sns

from tqdm import tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import pickle
import re
import json
from pathlib import Path


from open_spiel.python.algorithms.exploitability import nash_conv, best_response
from open_spiel.python.algorithms.ubc_exploitability import game_tree_size

from open_spiel.python.examples.ubc_plotting_utils import *
from open_spiel.python.examples.ubc_sample_game_tree import sample_game_tree, flatten_trees, flatten_tree
from open_spiel.python.examples.ubc_clusters import projectPCA, projectUMAP, fitGMM, projectTSNE
from open_spiel.python.examples.ubc_sample_game_tree import NodeType
from open_spiel.python.examples.ubc_evaluate_policy import run_eval

from auctions.webutils import *


from open_spiel.python.algorithms.random_agent import RandomAgent
import bokeh
from bokeh.layouts import row, column
from bokeh.plotting import figure, show, output_file, save
from bokeh.io import output_notebook
from bokeh.models import HoverTool, ColumnDataSource, ColorBar, LogColorMapper, LinearColorMapper
from bokeh.transform import linear_cmap, log_cmap
from bokeh.palettes import Category10_10, Magma256, Spectral10, Category20_20
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from collections import defaultdict
output_notebook()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Loading BokehJS ...

In [14]:
# GAME_NAME = 'sats_5.json'
# game = smart_load_clock_auction(GAME_NAME)
# game_config = game_config = load_game_config(GAME_NAME)
# config = read_config('april25/mlp')
# config['num_training_episodes'] = 100
# env_and_model = setup(game, game_config, config)
# # for i in range(game.num_players()):
# #     env_and_model.agents[i] = RandomAgent(i, game.num_distinct_actions())


In [16]:
# OPTIONS
EXPERIMENT_NAME = 'sats_games_5'
RUN_NAME = 'sats_5-april26mlp-100'
### Load checkpoint
checkpoint = get_checkpoint_by_name(EXPERIMENT_NAME, RUN_NAME)
game = load_game(checkpoint.game)
checkpoint.equilibrium_solver_run.config['device'] = 'cpu'
checkpoint.equilibrium_solver_run.config['cache_size'] = None
env_and_model = db_checkpoint_loader(checkpoint)
# for i in range(game.num_players()):
#     env_and_model.agents[i] = RandomAgent(i, game.num_distinct_actions())


Reading from env variable CLOCK_AUCTION_CONFIG_DIR. If it is not set, there will be trouble.
CLOCK_AUCTION_CONFIG_DIR=/apps/open_spiel/configs
Parsing configuration from /apps/open_spiel/configs/sats_5.json
Done config parsing
Reading from env variable CLOCK_AUCTION_CONFIG_DIR. If it is not set, there will be trouble.
CLOCK_AUCTION_CONFIG_DIR=/apps/open_spiel/configs
Parsing configuration from /apps/open_spiel/configs/sats_5.json
Done config parsing


In [21]:
def prof():
    return run_eval(env_and_model, 1000)

In [22]:
import cProfile, pstats
profiler = cProfile.Profile()
profiler.enable()
eval_output = prof()
profiler.disable()


In [23]:
pd.Series(eval_output['auction_lengths']).describe()

count    1000.000000
mean       20.965000
std         5.111938
min         3.000000
25%        17.750000
50%        22.000000
75%        25.000000
max        31.000000
dtype: float64

In [24]:
stats = pstats.Stats(profiler).sort_stats('cumtime')
stats.print_stats()


         7407514 function calls (7247196 primitive calls) in 28.301 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000   28.337   14.168 /home/ubuntu/.pyenv/versions/3.8.2/envs/venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3215(run_code)
        2    0.000    0.000   28.337   14.168 {built-in method builtins.exec}
        1    0.002    0.002   28.337   28.337 /tmp/ipykernel_8081/3371018378.py:4(<module>)
        1    0.369    0.369   28.335   28.335 /tmp/ipykernel_8081/816997627.py:1(prof)
        1    0.537    0.537   27.966   27.966 /apps/open_spiel/open_spiel/python/examples/ubc_evaluate_policy.py:36(run_eval)
    65895    0.543    0.000   22.718    0.000 /apps/open_spiel/open_spiel/python/examples/ubc_decorators.py:49(step)
    62193    0.508    0.000   20.402    0.000 /apps/open_spiel/open_spiel/python/pytorch/ubc_nfsp.py:218(step)
    59193    4.330    0.000   19.711    0

In [ ]:
# import pprofile
# profiler = pprofile.Profile()
# with profiler:
#     run_eval(env_and_model, 100)




In [ ]:
# profiler.print_stats()
# # profiler.dump_stats("profile.txt")


In [ ]:
state = env_and_model.game.new_initial_state()
state.apply_action(0)
state.apply_action(0)
state.apply_action(0)
len(state.information_state_tensor())

In [ ]:
env_and_model.agents[0]._rl_network